# Обработка диалогов Киа

In [ ]:
import pandas as pd
import requests
from io import BytesIO

# Шаг 1: Загрузить xlsx файл из Google Drive
google_sheet_url = "https://docs.google.com/spreadsheets/d/1OKS6LZNq9AZo_WkGw9y6jb_o7VTGUbl_rX329vLQEz0/export?format=xlsx"
response = requests.get(google_sheet_url)
data = pd.read_excel(BytesIO(response.content))

# Шаг 2: Извлечь столбец "text"
text_column = data["text"]

# Шаг 3: Заменить "operatorMessage: Здравствуйте" на "<operatorMessage: Здравствуйте>"
text_column = text_column.str.replace("operatorMessage: Здравствуйте", "<operatorMessage: Здравствуйте>")

# Шаг 4: Сохранить в файл "dialog.txt" в корне Colab
with open("/content/kia_dialog.txt", "w", encoding="utf-8") as f:
    for text in text_column:
        f.write(str(text) + "\n")

print("Файл kia_dialog.txt успешно сохранен!")


In [ ]:
!pip install openai tiktoken langchain faiss-cpu

In [ ]:
import tiktoken
import getpass
import openai
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

## Первый вариант подсчета токенов:

In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Возвращает количество токенов, используемых списком сообщений."""
    try:
        encoding = tiktoken.encoding_for_model(model) # Пытаемся получить кодировку для выбранной модели
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base") # если не получается, используем кодировку "cl100k_base"
    if model == "gpt-3.5-turbo-0301" or "gpt-3.5-turbo-0613" or "gpt-3.5-turbo-16k" or "gpt-3.5-turbo":
        num_tokens = 0 # начальное значение счетчика токенов
        for message in messages: # Проходимся по каждому сообщению в списке сообщений
            num_tokens += 4  # каждое сообщение следует за <im_start>{role/name}\n{content}<im_end>\n, что равно 4 токенам
            for key, value in message.items(): # итерация по элементам сообщения (роль, имя, контент)
                num_tokens += len(encoding.encode(value)) # подсчет токенов в каждом элементе
                if key == "name":  # если присутствует имя, роль опускается
                    num_tokens += -1  # роль всегда требуется и всегда занимает 1 токен, так что мы вычитаем его, если имя присутствует
        num_tokens += 2  # каждый ответ начинается с <im_start>assistant, что добавляет еще 2 токена
        return num_tokens # возвращаем общее количество токенов
    else:
      # Если выбранная модель не поддерживается, генерируем исключение
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}. # вызываем ошибку, если функция не реализована для конкретной модели""")

In [ ]:
system = ''' '''

base_info= '''Основной контекст'''

question = "Суть вопроса"

In [ ]:
messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": base_info+ f"{question}" }
      ]


In [ ]:
print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} токенов использовано на вопрос")

## Второй вариант подсчета токенов:

In [ ]:
messages = [
    SystemMessage(content=system),
    HumanMessage(content=base_info+ f"{question}")
    ]

In [ ]:
chat = ChatOpenAI(temperature=0)
chat.get_num_tokens_from_messages(messages)

Загрузим текст из файла kia_dialog.txt и разделим на отрезки(chunks)

In [ ]:
loader = TextLoader("/content/kia_dialog.txt")
documents = loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(separator="<", chunk_size=10000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
print(docs[0])

In [ ]:
len(docs)

In [ ]:
type(documents)

In [ ]:
type(docs)

In [ ]:
messages = [
    SystemMessage(content=system),
    HumanMessage(content=f"{docs[0]}")
    ]

In [ ]:
chat.get_num_tokens_from_messages(messages)